In [1]:
import csv
import numpy as np
import pandas as pd
import math
import mpu
from datetime import datetime,time,timedelta

In [2]:
#Read PlayCar Data
PlayCar=pd.read_csv("/Users/samiraziyadidegan/Documents/Carsharing_Luca/Spostamenti_Casa_Lavoro.csv")
PlayCar.head()

,N,DAY_START,START,DAY_END,END,LAT_HOME,LON_HOME,LAT_DEST,LON_DEST
0,1,1/1/18,8:45:00,1/1/18,17:45:00,39.223491,9.129467,39.221950,9.102410
1,2,1/1/18,7:15:00,1/1/18,16:10:00,39.210025,9.138713,39.214436,9.119890
2,3,1/1/18,8:15:00,1/1/18,17:14:00,39.199243,9.140444,39.215199,9.110728
3,4,1/1/18,7:30:00,1/1/18,16:30:00,39.219507,9.112696,39.211578,9.119600
4,5,1/1/18,7:45:00,1/1/18,15:40:00,39.199339,9.149289,39.211641,9.119082


In [3]:
#Merge Date and Time
PlayCar['Start_Datetime']=PlayCar['DAY_START'] + str(' ') + PlayCar['START']
PlayCar['End_Datetime']=PlayCar['DAY_END'] + str(' ') + PlayCar['END']
PlayCar['Start_Datetime']=pd.to_datetime(PlayCar['Start_Datetime'])
PlayCar['End_Datetime']=pd.to_datetime(PlayCar['End_Datetime'])
PlayCar.head()

,N,DAY_START,START,DAY_END,END,LAT_HOME,LON_HOME,LAT_DEST,LON_DEST,Start_Datetime,End_Datetime
0,1,1/1/18,8:45:00,1/1/18,17:45:00,39.223491,9.129467,39.221950,9.102410,2018-01-01 08:45:00,2018-01-01 17:45:00
1,2,1/1/18,7:15:00,1/1/18,16:10:00,39.210025,9.138713,39.214436,9.119890,2018-01-01 07:15:00,2018-01-01 16:10:00
2,3,1/1/18,8:15:00,1/1/18,17:14:00,39.199243,9.140444,39.215199,9.110728,2018-01-01 08:15:00,2018-01-01 17:14:00
3,4,1/1/18,7:30:00,1/1/18,16:30:00,39.219507,9.112696,39.211578,9.119600,2018-01-01 07:30:00,2018-01-01 16:30:00
4,5,1/1/18,7:45:00,1/1/18,15:40:00,39.199339,9.149289,39.211641,9.119082,2018-01-01 07:45:00,2018-01-01 15:40:00


In [4]:
#Sort the data based on both start and end time
PlayCar=PlayCar.sort_values(by=['Start_Datetime', 'End_Datetime'])
PlayCar=PlayCar.reset_index(drop=True)
PlayCar=PlayCar.reset_index()
PlayCar=PlayCar.drop(columns=['N'])
#PlayCar=PlayCar.drop(columns=['N','X_HOME','Y_HOME','X_DEST','Y_DEST'])
PlayCar=PlayCar.rename(columns={"index": "User_Code"})
PlayCar.head()

,User_Code,DAY_START,START,DAY_END,END,LAT_HOME,LON_HOME,LAT_DEST,LON_DEST,Start_Datetime,End_Datetime
0,0,1/1/18,0:00:00,1/1/18,10:50:00,39.208740,9.127565,39.224223,9.112245,2018-01-01 00:00:00,2018-01-01 10:50:00
1,1,1/1/18,0:15:00,1/1/18,8:00:00,39.247468,9.121087,39.222414,9.103606,2018-01-01 00:15:00,2018-01-01 08:00:00
2,2,1/1/18,0:15:00,1/1/18,8:00:00,39.219485,9.125207,39.223841,9.121661,2018-01-01 00:15:00,2018-01-01 08:00:00
3,3,1/1/18,0:15:00,1/1/18,9:00:00,39.255991,9.127585,39.248423,9.133761,2018-01-01 00:15:00,2018-01-01 09:00:00
4,4,1/1/18,0:15:00,1/1/18,9:00:00,39.221169,9.109922,39.245405,9.135553,2018-01-01 00:15:00,2018-01-01 09:00:00


In [5]:
Station=pd.read_csv("/Users/samiraziyadidegan/Documents/Carsharing_Luca/Stations_3.csv")
Station=Station.sort_values(by=['Vehicles', 'Station ID'])
Station=Station.reset_index()
Station=Station.drop(columns=['index'])

Station.head()

,Station ID,Latitude,Longitude,Vehicles
0,1,39.22304,9.12341,3
1,2,39.22706,9.12667,3
2,3,39.23932,9.12917,3
3,4,39.22029,9.11567,3
4,5,39.20682,9.13824,3


In [6]:
print(min(PlayCar.Start_Datetime))
print(max(PlayCar.End_Datetime))

2018-01-01 00:00:00
2018-01-02 06:30:00


In [7]:
Steps=[]
ts=datetime(2018,1,1,0,0,0)
while ts < datetime (2018,1,2,7,0,0):
    Steps.append(datetime(ts.year, ts.month, ts.day, ts.hour, ts.minute, ts.second))
    ts += timedelta(minutes=15)

In [8]:
j=0
i_New=0
#Critical_Distance
Distance=500
#Critivcal Time
Critical_Time=3600
Num_Station=Station.shape[0]
Speed=20
Parking_Set=0
Parking_User_Assign = [None]*PlayCar.shape[0]
Parking_User_Location_X=[None]*PlayCar.shape[0]
Parking_User_Location_Y=[None]*PlayCar.shape[0]
Criteria22=[99999]
Criteria222=[99999]
Station_User_X=[None]*PlayCar.shape[0]
Station_User_Y=[None]*PlayCar.shape[0]

Booking_Result=[None]*PlayCar.shape[0]
Results=[None]*PlayCar.shape[0]
Reference=[999999]*PlayCar.shape[0]

In [9]:
for t in Steps:
    #print("t: ", t)
    for i in range(PlayCar.shape[0]): 
        
        Criteria1=[99999]*Num_Station
        Criteria11=[99999]*Num_Station

        Sorted_Criteria1=[]
        Sorted_Criteria11=[]
        Sorted_Criteria22=[]
        
        if PlayCar.Start_Datetime[i] >= t and PlayCar.Start_Datetime[i] < t+timedelta(minutes=15):
            #print("0")
            #print("Parking_Set: ",Parking_Set)
            #print(Station.Vehicles > 0)
            if Parking_Set == 0 and (Station.Vehicles > 0).any():
                #print("1")
                for n in range(Num_Station):
                    Criteria1[n] = mpu.haversine_distance((PlayCar.LAT_HOME[i],PlayCar.LON_HOME[i]), (Station.Latitude[n],Station.Longitude[n]))
                #print("home to station 1: ", Criteria1)
                Sorted_Criteria1 = sorted(Criteria1)
                s = np.array(Criteria1) 
                index1 = np.argsort(s)
                           
                for m in range(Num_Station):
                    if Sorted_Criteria1[m] <= Distance/1000:
                        #print("one")
                        Booking_Result[i]="Accept"
                        Station.Vehicles[index1[m]] = Station.Vehicles[index1[m]]-1
                        Parking_Set=Parking_Set+1
                                
                        #Parking_User_Assign.append(None)
                        #Parking_User_Location_X.append(None)
                        #Parking_User_Location_Y.append(None)
                                
                        Parking_User_Assign[j] = i
                        Parking_User_Location_X[j] = PlayCar.LAT_DEST [i]
                        Parking_User_Location_Y[j] = PlayCar.LON_DEST [i]
                        Station_User_X[i]=Station.Latitude[index1[m]]
                        Station_User_Y[i]=Station.Longitude[index1[m]]                       
                        Results[i]="Station Vehicle is Assigned" 
                        #print("j: ",j)
                        #print("Parking_User_Location_X[j]", Parking_User_Location_X[j])
                        #print("Parking_User_Location_Y[j]", Parking_User_Location_Y[j])
                        #print("Station number: ", index1[m])
                        #print("user: ", i)
                        #print(Results[i])
                        #print("Parking_Set: ",Parking_Set)
                        Reference[i]=index1[m] 
                        break
                        
                    elif m==Num_station-1 and Sorted_Criteria1[0] > Distance/1000:
                        #print("two")
                        Booking_Result[i]="Reject"
                        Results[i]="Long Walking Distance"
                        #print(Results[i])
                        #print("Parking_Set: ",Parking_Set)
                        Reference[i]=999999
                        
                 
            elif Parking_Set > 0 and (Station.Vehicles > 0).any():
                #print("2")
                for n in range(Num_Station):
                    if Station.Vehicles[n] > 0:
                        Criteria11[n] = mpu.haversine_distance((PlayCar.LAT_HOME[i],PlayCar.LON_HOME[i]), (Station.Latitude[n],Station.Longitude[n]))
                    else:
                        Criteria11[n] = 99999 
                #print("home to station 2: ", Criteria11)     
                Sorted_Criteria11 = sorted(Criteria11)
                s = np.array(Criteria11) 
                index2 = np.argsort(s)
            
                for j in range (j+1):
                    if (PlayCar.End_Datetime[Parking_User_Assign[j]].timestamp()-PlayCar.Start_Datetime[Parking_User_Assign[j]].timestamp()) >= Critical_Time and \
                    (PlayCar.Start_Datetime[Parking_User_Assign[j]].timestamp()+ mpu.haversine_distance((Station_User_X[Parking_User_Assign[j]],Station_User_Y[Parking_User_Assign[j]]), (PlayCar.LAT_DEST[j],PlayCar.LON_DEST[j]))/Speed*3600) <= PlayCar.Start_Datetime[i].timestamp() and \
                    (PlayCar.End_Datetime[Parking_User_Assign[j]].timestamp()- mpu.haversine_distance((Station_User_X[Parking_User_Assign[j]],Station_User_Y[Parking_User_Assign[j]]),(PlayCar.LAT_DEST[j],PlayCar.LON_DEST[j]))/Speed*3600) >= PlayCar.End_Datetime[i].timestamp():
                        Criteria22[j]=mpu.haversine_distance((PlayCar.LAT_HOME[i],PlayCar.LON_HOME[i]), (Parking_User_Location_X[j],Parking_User_Location_Y[j]))
                #print("home to parking 2:", Criteria22)
                Sorted_Criteria22=sorted(Criteria22)
                s = np.array(Criteria22) 
                index3 = np.argsort(s)
               
                if Sorted_Criteria11[0] >= Sorted_Criteria22[0] and Sorted_Criteria22[0] <= Distance/1000:
                    #print("three")
                    Booking_Result[i]="Accept"
                    Parking_User_Assign[index3[0]] = i 
                    Parking_User_Location_X[index3[0]] = PlayCar.LAT_DEST [i] 
                    Parking_User_Location_Y[index3[0]] = PlayCar.LON_DEST [i] 
                    Station_User_X[i]=PlayCar.LAT_DEST [index3[0]] 
                    Station_User_Y[i]=PlayCar.LON_DEST [index3[0]] 
                    Reference[i]=index3[0]
                    #print("j: ",j)
                    #print("Parking_User_Location_X[j]", Parking_User_Location_X[index3[0]])
                    #print("Parking_User_Location_Y[j]", Parking_User_Location_Y[index3[0]])
                    Results[i]="Parked Vehicle is Assigned"
                    #print("User: ", i)
                    #print("parking_User index", index3[0])
                    #print(Results[i])
                    #print("Parking_Set: ",Parking_Set )
                   
                elif  Station.Vehicles[index2[0]]>0 and Sorted_Criteria11[0] <= Distance/1000:
                    #print("four")
                    Booking_Result[i]="Accept"
                    Station.Vehicles[index2[0]] = Station.Vehicles[index2[0]] - 1
                    Parking_Set=Parking_Set+1 
                    j=j+1
                        
                    Criteria22.append(99999)
                    Criteria222.append(99999)
                    #Parking_User_Assign.append(None)
                    #Parking_User_Location_X.append(None)
                    #Parking_User_Location_Y.append(None)
                        
                    Parking_User_Assign[j] = i 
                    Parking_User_Location_X[j] = PlayCar.LAT_DEST [i] 
                    Parking_User_Location_Y[j] = PlayCar.LON_DEST [i] 
                    Station_User_X[i]=Station.Latitude[index2[0]] 
                    Station_User_Y[i]=Station.Longitude[index2[0]] 
                    Results[i]="Station Vehicle is Assigned" 
                    #print("j: ",j)
                    #print("Parking_User_Location_X[j]", Parking_User_Location_X[j])
                    #print("Parking_User_Location_Y[j]", Parking_User_Location_Y[j])
                    #print("Station number: ", index2[0])
                    #print("Parking_Set: ",Parking_Set )
                    #print("user: ", i)
                    #print(Results[i])
                    Reference[i]=index2[0]
                    
                elif   Sorted_Criteria11[0] > Distance/1000:
                    #print("five")
                    Booking_Result[i]="Reject" 
                    Reference[i]=999999
                    #print("User: ", i)
                    Results[i]="Long Walking Distance"
                    #print("Parking_Set: ",Parking_Set )
                    #print(Results[i])
                                                
                elif   (Sorted_Criteria22==99999).all() and np.all(Station.Vehicles==0):
                    #print("six")
                    Booking_Result[i]="Reject" 
                    Reference[i]=999999
                    Results[i]="No Available Vehicle"
                    #print("User: ", i)
                    #print(Results[i])
                    #print("Parking_Set: ",Parking_Set )
                                    
                                  
            elif   Parking_Set >0 and np.all(Station.Vehicles==0):
                #print("3")
                for j in range(j+1):  
                    Criteria222[j]=mpu.haversine_distance((PlayCar.LAT_HOME[i],PlayCar.LON_HOME[i]),(Parking_User_Location_X[j],Parking_User_Location_Y[j]))
                #print("home to parking 3:", Criteria222)
                Sorted_Criteria222=sorted(Criteria222)
                s = np.array(Criteria222) 
                index4 = np.argsort(s)
                
                for k in range(j+1):
                    if (PlayCar.End_Datetime[Parking_User_Assign[index4[k]]].timestamp()-PlayCar.Start_Datetime[Parking_User_Assign[index4[k]]].timestamp()) >= Critical_Time and \
                    (PlayCar.Start_Datetime[Parking_User_Assign[index4[k]]].timestamp()+mpu.haversine_distance((Station_User_X[Parking_User_Assign[index4[k]]],Station_User_Y[Parking_User_Assign[index4[k]]]), (PlayCar.LAT_DEST[index4[k]],PlayCar.LON_DEST[index4[k]]))/Speed*3600) <= PlayCar.Start_Datetime[i].timestamp() and \
                    (PlayCar.End_Datetime[Parking_User_Assign[index4[k]]].timestamp()-mpu.haversine_distance((Station_User_X[Parking_User_Assign[index4[k]]],Station_User_Y[Parking_User_Assign[index4[k]]]),(PlayCar.LAT_DEST[index4[k]],PlayCar.LON_DEST[index4[k]]))/Speed*3600) >= PlayCar.End_Datetime[i].timestamp() and \
                    Sorted_Criteria222[k] <= Distance/1000:
                        print("seven")
                        Booking_Result[i]="Accept"
                        Parking_User_Assign[index4[k]] = i 
                        Parking_User_Location_X[index4[k]] = PlayCar.LAT_DEST [i] 
                        Parking_User_Location_Y[index4[k]] = PlayCar.LON_DEST [i] 
                        Station_User_X[i]=PlayCar.LAT_DEST [index4[k]] 
                        Station_User_Y[i]=PlayCar.LON_DEST [index4[k]] 
                        Reference[i]=index4[k]
                        Results[i]="Parked Vehicle is Assigned"
                        #print("j: ",j)
                        #print("Parking_User_Location_X[index4[k]]", Parking_User_Location_X[index4[k]])
                        #print("Parking_User_Location_Y[index4[k]]", Parking_User_Location_Y[index4[k]])
                        #print("parking_User index", index4[k])
                        #print("Parking_Set: ",Parking_Set )
                        #print("User: ", i)
                        #print(Results[i])
                        break
                  
                    else:
                        #print("eight")
                        Booking_Result[i] ="Reject"
                        #print("User: ", i)
                        Results[i]="No Available Vehicle at Stations and Incompatible Parked Vehicle Reserved Time" 
                        Reference[i]=999999 
                        #print("Parking_Set: ",Parking_Set )
                        #print(Results[i])
                                
            
            elif   Parking_Set ==0 and np.all(Station.Vehicles==0):
                #print("4")
                #print("nine")
                Booking_Result[i] ="Reject"
                #print("User: ", i)
                Results[i]="No Available Vehicle"
                #print(Results[i])
                #print("Parking_Set: ",Parking_Set )

    
        if t <= PlayCar.End_Datetime[i] and PlayCar.End_Datetime[i]< t+timedelta(minutes=15):
            if Booking_Result[i] == "Accept":
                if Results[i]=="Station Vehicle is Assigned":
                    Station.Vehicles[Reference[i]] = Station.Vehicles[Reference[i]] + 1
                    parking_Set = Parking_Set-1
                    j=j-1
                    Parking_User_Location_X[Reference[i]]=-90   
                    Parking_User_Location_Y[Reference[i]]=-90   
                    #print("return1: ",Reference[i])   
                    
                    
                elif Results[i]=="Parked Vehicle is Assigned":
                    Parking_User_Assign[i] = Reference[i] 
                    Parking_User_Location_X[i] = PlayCar.LAT_DEST [Reference[i]] 
                    Parking_User_Location_Y[i] = PlayCar.LON_DEST [Reference[i]] 
                    Station_User_X[Reference[i]]=PlayCar.LAT_DEST [i] 
                    Station_User_Y[Reference[i]]=PlayCar.LON_DEST [i] 
                    #print("return2: ",Reference[i] )    

/Users/samiraziyadidegan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/samiraziyadidegan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/samiraziyadidegan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
PlayCar['Booking_Result']=Booking_Result
PlayCar['Results']=Results
PlayCar.head(2)

,User_Code,DAY_START,START,DAY_END,END,LAT_HOME,LON_HOME,LAT_DEST,LON_DEST,Start_Datetime,End_Datetime,Booking_Result,Results
0,0,1/1/18,0:00:00,1/1/18,10:50:00,39.208740,9.127565,39.224223,9.112245,2018-01-01 00:00:00,2018-01-01 10:50:00,Accept,Station Vehicle is Assigned
1,1,1/1/18,0:15:00,1/1/18,8:00:00,39.247468,9.121087,39.222414,9.103606,2018-01-01 00:15:00,2018-01-01 08:00:00,Accept,Station Vehicle is Assigned


In [11]:
PlayCar.to_csv("/Users/samiraziyadidegan/Documents/Carsharing_Luca/Speed_20/Casa_Lavoro_Sublease_500_3.csv")